In [1]:
import tensorflow as tf
import os
import sys
import data_generation
import networks
import scipy.io as sio
import param
import util
import truncated_vgg
from keras.backend.tensorflow_backend import set_session
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
def train(model_name, gpu_id):
    params = param.get_general_params()

    network_dir = params['model_save_dir'] + '/' + model_name

    if not os.path.isdir(network_dir):
        os.mkdir(network_dir)

    train_feed = data_generation.create_feed(params, params['data_dir'], 'train')

    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.Session(config=config))

    vgg_model = truncated_vgg.vgg_norm()
    networks.make_trainable(vgg_model, False)
    response_weights = sio.loadmat('../data/vgg_activation_distribution_train.mat')
    model = networks.network_posewarp(params)
    model.compile(optimizer=Adam(lr=1e-4), loss=[networks.vgg_loss(vgg_model, response_weights, 12)])

    #model.summary()
    n_iters = params['n_training_iter']

    for step in range(0, n_iters):
        x, y = next(train_feed)

        train_loss = model.train_on_batch(x, y)

        util.printProgress(step, 0, train_loss)

        if step > 0 and step % params['model_save_interval'] == 0:
            model.save(network_dir + '/' + str(step) + '.h5')

In [3]:
params = param.get_general_params()

In [4]:
params

{'IMG_HEIGHT': 256,
 'IMG_WIDTH': 256,
 'batch_size': 1,
 'data_dir': '/mnt/data/jarvislam1999/posewarp-cvpr2018/data',
 'limbs': [[0, 1],
  [2, 3],
  [3, 4],
  [5, 6],
  [6, 7],
  [8, 9],
  [9, 10],
  [11, 12],
  [12, 13],
  [2, 5, 8, 11]],
 'max_px_shift': 10,
 'max_rotate_degree': 5,
 'max_sat_factor': 0.05,
 'model_save_dir': '/mnt/data/jarvislam1999/posewarp-cvpr2018/models',
 'model_save_interval': 1000,
 'n_joints': 14,
 'n_limbs': 10,
 'n_training_iter': 200000,
 'obj_scale_factor': 1.14,
 'posemap_downsample': 2,
 'project_dir': '/mnt/data/jarvislam1999/posewarp-cvpr2018',
 'scale_max': 1.05,
 'scale_min': 0.9,
 'sigma_joint': 1.75,
 'test_interval': 500}

In [5]:
params = param.get_general_params()

In [6]:
model_name = 'vgg_100000.h5'

In [7]:
network_dir = params['model_save_dir'] + '/' + model_name

In [8]:
network_dir

'/mnt/data/jarvislam1999/posewarp-cvpr2018/models/vgg_100000.h5'

In [9]:
params['data_dir']

'/mnt/data/jarvislam1999/posewarp-cvpr2018/data'

In [10]:
os.path.isdir(params['model_save_dir'])

True

In [11]:
if not os.path.isdir(params['model_save_dir']):
    os.mkdir(params['model_save_dir'])

In [12]:
train_feed = data_generation.create_feed(params, params['data_dir'], 'train')

In [13]:
train_feed

<generator object warp_example_generator at 0x7f9319437c50>

In [14]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [15]:
vgg_model = truncated_vgg.vgg_norm()
networks.make_trainable(vgg_model, False)
response_weights = sio.loadmat('../data/vgg_activation_distribution_train.mat')
model = networks.network_posewarp(params)
model.compile(optimizer=Adam(lr=1e-4), loss=[networks.vgg_loss(vgg_model, response_weights, 12)])

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
#model.summary()
n_iters = params['n_training_iter']

for step in range(0, n_iters):
    x, y = next(train_feed)

    train_loss = model.train_on_batch(x, y)

    util.printProgress(step, 0, train_loss)

    if step > 0 and step % params['model_save_interval'] == 0:
        model.save(network_dir + '/' + str(step) + '.h5')

Instructions for updating:
Use tf.cast instead.


/home/jarvislam1999/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
0,0,0.401758
1,0,0.396321
2,0,0.383585
3,0,0.38894
4,0,0.372299
5,0,0.359971
6,0,0.388343
7,0,0.37485
8,0,0.382665
9,0,0.352024
10,0,0.365384
11,0,0.364588
12,0,0.329552
13,0,0.385734
14,0,0.363205
15,0,0.320669
16,0,0.369317
17,0,0.325524
18,0,0.346348
19,0,0.317351
20,0,0.326536
21,0,0.343946
22,0,0.327916
23,0,0.330689
24,0,0.324959
25,0,0.353021
26,0,0.34472
27,0,0.34183
28,0,0.328171
29,0,0.309264
30,0,0.275217
31,0,0.290652
32,0,0.300195
33,0,0.296611
34,0,0.284229
35,0,0.327151
36,0,0.343696
37,0,0.31312
38,0,0.328172
39,0,0.258299
40,0,0.293144
41,0,0.303723
42,0,0.330986
43,0,0.274078
44,0,0.283903
45,0,0.264392
46,0,0.268838
47,0,0.233552
48,0,0.320612
49,0,0.289181
50,0,0.284629
51,0,0.29228


KeyboardInterrupt: 

In [18]:
from keras.applications.vgg19 import VGG19
from keras.layers import AveragePooling2D, Conv2D, Input
from keras.models import Model

In [ ]:
img_input = Input(shape=(256, 256, 3))

x1 = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x2 = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x1)
x3 = AveragePooling2D((2, 2), strides=(2, 2), name='block1_pool')(x2)

x4 = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x3)
x5 = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x4)
x6 = AveragePooling2D((2, 2), strides=(2, 2), name='block2_pool')(x5)

x7 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x6)
x8 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x7)
x9 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x8)
x10 = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x9)
x11 = AveragePooling2D((2, 2), strides=(2, 2), name='block3_pool')(x10)

x12 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x11)
x13 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x12)
x14 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x13)
x15 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x14)
x16 = AveragePooling2D((2, 2), strides=(2, 2), name='block4_pool')(x15)

x17 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x16)
x18 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x17)
x19 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x18)
x20 = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x19)
x21 = AveragePooling2D((2, 2), strides=(2, 2), name='block5_pool')(x20)

model = Model(inputs=[img_input], outputs=[x1, x2, x4, x5, x7, x8, x9, x10, x12, x13, x14, x15])
model_orig = VGG19(weights='imagenet', input_shape=(256, 256, 3), include_top=False)

'''
for i in range(len(model.layers)):
    weights = model_orig.layers[i].get_weights()
    model.layers[i].set_weights(weights)
'''